In [ ]:
!nvidia-smi


Thu Aug 22 09:31:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Standard libraries
import sys  # System-specific parameters and functions
import os   # Miscellaneous operating system interfaces
import warnings  # Warning control
warnings.filterwarnings("ignore")

# Data manipulation
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical operations

# Visualization
import matplotlib.pyplot as plt  # Plotting library
import seaborn as sns  # Statistical data visualization
from matplotlib.colors import ListedColormap  # Colormap utilities

# Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler  # Preprocessing tools
from sklearn import model_selection, metrics, preprocessing  # Model selection, evaluation, and preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV  # Model selection and evaluation
from sklearn.tree import DecisionTreeClassifier  # Decision tree classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier  # Ensemble methods
from xgboost import XGBClassifier  # Extreme Gradient Boosting
from sklearn.neighbors import KNeighborsClassifier  # k-Nearest Neighbors
from sklearn.neural_network import MLPClassifier  # Multi-layer Perceptron

# Statistical analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor  # Variance inflation factor
from scipy.stats import pointbiserialr, chi2_contingency, spearmanr, entropy  # Statistical functions
from statsmodels.graphics.gofplots import qqplot  # Q-Q plot
from collections import Counter  # Container datatypes

# Tabulate
from tabulate import tabulate  # Pretty-print tabular data

# Set visualization style
#sns.set()  # Set Seaborn default style
#plt.style.use('ggplot')  # Set ggplot style for matplotlib

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

# Balancing techniques
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

In [ ]:

from google.colab import files


uploaded = files.upload()

Saving encoded_df.csv to encoded_df.csv


In [ ]:
#import encoded dataset
import io

encoded_df = pd.read_csv(io.BytesIO(uploaded['encoded_df.csv']))
encoded_df.head()

,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,claim_day_of_week,...,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,Month,Day,Year
0,46,1,1.0,85,38301,1,1,1,80006,4,...,7530.940993,9.0,0,12885.45235,6,16161.33381,0,12,16,2016
1,21,0,0.0,75,30445,0,1,1,15021,3,...,2966.024895,4.0,2,29429.45218,6,28691.96422,0,2,12,2015
2,49,0,0.0,87,38923,0,1,0,20158,1,...,6283.888333,3.0,0,21701.18195,6,22090.94758,1,12,6,2016
3,58,0,1.0,58,40605,1,0,0,15024,3,...,6169.747994,4.0,1,13198.27344,3,38329.58106,1,5,5,2016
4,38,1,1.0,95,36380,1,0,1,50034,1,...,4541.387150,7.0,1,38060.21122,2,25876.56319,0,10,27,2015


In [ ]:
X=encoded_df.drop(columns=['fraud'])
y=encoded_df['fraud']

#20% allocated for test data and 80% for train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=28)

In [ ]:
train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [ ]:
continuous_vars = encoded_df.select_dtypes(include=['int64', 'float64']).columns
Continuous_Variables_in_X_train = continuous_vars

categorical_vars = encoded_df.select_dtypes(include=['object', 'category']).columns
Categorical_Variables_in_X_train = [var for var in categorical_vars if var != 'fraud']

Date_Month_Year= encoded_df.select_dtypes(include=['int32']).columns

# Display the variables
print("Continuous Variables in X_train:")
print(Continuous_Variables_in_X_train)
print("\nCategorical Variables in X train:")
print(Categorical_Variables_in_X_train)
print("\nDate_Month_Year:")
print(Date_Month_Year)

Continuous Variables in X_train:
Index(['age_of_driver', 'gender', 'marital_status', 'safty_rating',
       'annual_income', 'high_education_ind', 'address_change_ind',
       'living_status', 'zip_code', 'claim_day_of_week', 'accident_site',
       'past_num_of_claims', 'witness_present_ind', 'liab_prct', 'channel',
       'policy_report_filed_ind', 'claim_est_payout', 'age_of_vehicle',
       'vehicle_category', 'vehicle_price', 'vehicle_color', 'vehicle_weight',
       'fraud', 'Month', 'Day', 'Year'],
      dtype='object')

Categorical Variables in X train:
[]

Date_Month_Year:
Index([], dtype='object')


#### checking imbalance of response

In [ ]:
print ('Total not committed fraud :  {} and its percentage is {} %'.format(encoded_df.fraud.value_counts()[0], round(encoded_df.fraud.value_counts()[0]/encoded_df.fraud.value_counts().sum()*100,2)) )
print ('Total committed fraud :  {} and its percentage is {} %'.format(encoded_df.fraud.value_counts()[1], round(encoded_df.fraud.value_counts()[1]/encoded_df.fraud.value_counts().sum()*100,2)) )

Total not committed fraud :  15040 and its percentage is 84.32 %
Total committed fraud :  2796 and its percentage is 15.68 %


## --------------------------------------------------------------------------------------------------------

In [ ]:
df_backup = encoded_df.copy()
df_backup.head()

,age_of_driver,gender,marital_status,safty_rating,annual_income,high_education_ind,address_change_ind,living_status,zip_code,claim_day_of_week,...,claim_est_payout,age_of_vehicle,vehicle_category,vehicle_price,vehicle_color,vehicle_weight,fraud,Month,Day,Year
0,46,1,1.0,85,38301,1,1,1,80006,4,...,7530.940993,9.0,0,12885.45235,6,16161.33381,0,12,16,2016
1,21,0,0.0,75,30445,0,1,1,15021,3,...,2966.024895,4.0,2,29429.45218,6,28691.96422,0,2,12,2015
2,49,0,0.0,87,38923,0,1,0,20158,1,...,6283.888333,3.0,0,21701.18195,6,22090.94758,1,12,6,2016
3,58,0,1.0,58,40605,1,0,0,15024,3,...,6169.747994,4.0,1,13198.27344,3,38329.58106,1,5,5,2016
4,38,1,1.0,95,36380,1,0,1,50034,1,...,4541.387150,7.0,1,38060.21122,2,25876.56319,0,10,27,2015


## Downsampled

In [ ]:
X_backup = df_backup.drop(columns=['fraud'])
y_backup = df_backup['fraud']

#20% allocated for test data and 80% for train data
X_train_backup, X_test_backup, y_train_backup, y_test_backup = train_test_split(X_backup, y_backup, test_size=0.2, random_state=28)

In [ ]:
#separate majority and minority classes
majority_class = X_train_backup[y_train_backup == 0]
minority_class = X_train_backup[y_train_backup == 1]

print("Size of majority class before upsampling:", majority_class.shape[0])
print("Size of minority class before upsampling:", minority_class.shape[0])

Size of majority class before upsampling: 12036
Size of minority class before upsampling: 2232


In [ ]:
#downsample majority class
majority_downsampled = resample(majority_class,
replace=False,
n_samples=len(minority_class),
random_state=28)

X_downsampled = np.vstack([majority_downsampled, minority_class])
y_downsampled = np.concatenate([np.zeros(len(minority_class)), np.ones(len(minority_class))])

#shuffle data
shuffle_indices = np.arange(len(X_downsampled))
np.random.shuffle(shuffle_indices)
X_downsampled = X_downsampled[shuffle_indices]
y_downsampled = y_downsampled[shuffle_indices]

unique_classes, class_counts = np.unique(y_downsampled, return_counts=True)
print("Class counts after downsampling:")
for cls, count in zip(unique_classes, class_counts):
  print(f"Class {int(cls)}: {count}")



Class counts after downsampling:
Class 0: 2232
Class 1: 2232


### Logistic Regression

In [ ]:
lr = LogisticRegression()
result = lr.fit(X_downsampled,y_downsampled)
y_pred = lr.predict(X_test)

accuracy = round(metrics.accuracy_score(y_test, y_pred)*100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred)*100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  57.43 %
Misclassification rate of this model:  42.57 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.873     0.579     0.696      3004
           1      0.197     0.551     0.291       564

    accuracy                          0.574      3568
   macro avg      0.535     0.565     0.493      3568
weighted avg      0.766     0.574     0.632      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1738 |                 1266 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  253 |                  311 |
+-----------------+----------------------+----------------------+


### Decision Trees

In [ ]:
dt = DecisionTreeClassifier(random_state=28)
param_grid = {
    'max_depth': [3, 4, 5, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto', 'log2',None]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=dt, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_downsampled, y_downsampled)

# Print the best hyperparameters
print(grid_search.best_params_)

# Print the best score
print(grid_search.best_score_)

{'max_depth': 4, 'max_features': None, 'min_samples_leaf': 3, 'min_samples_split': 2}
0.6176123210420862


In [ ]:
dtree = DecisionTreeClassifier(max_depth = 4,  min_samples_split = 2, min_samples_leaf = 3, max_features = None, random_state = 28)
dtree.fit(X_downsampled,y_downsampled)
y_pred = dtree.predict(X_test)

accuracy = round(metrics.accuracy_score(y_test, y_pred)*100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred)*100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  59.89 %
Misclassification rate of this model:  40.11 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.908     0.583     0.710      3004
           1      0.236     0.686     0.351       564

    accuracy                          0.599      3568
   macro avg      0.572     0.634     0.530      3568
weighted avg      0.802     0.599     0.653      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1750 |                 1254 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  177 |                  387 |
+-----------------+----------------------+----------------------+


### Random Forest

In [ ]:
np.random.seed(28)

rf = RandomForestClassifier(random_state=28)
param_grid = {
    'n_estimators': [50, 80, 100, 120, 150],
    'max_depth': [3, 4, 5, None],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['auto', 'log2',None]
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the GridSearchCV object to the data
grid_search.fit(X_downsampled, y_downsampled)

# Print the best hyperparameters
print(grid_search.best_params_)

# Print the best score
print(grid_search.best_score_)

{'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 80}
0.6402389788037502


In [ ]:
rf = RandomForestClassifier(n_estimators=80, max_depth=4, max_features='log2', min_samples_leaf=3, min_samples_split=2, random_state=28)
rf.fit(X_downsampled,y_downsampled)
y_pred = rf.predict(X_test)

accuracy = round(metrics.accuracy_score(y_test, y_pred)*100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred)*100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  54.68 %
Misclassification rate of this model:  45.32 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.912     0.511     0.655      3004
           1      0.221     0.738     0.340       564

    accuracy                          0.547      3568
   macro avg      0.566     0.624     0.497      3568
weighted avg      0.803     0.547     0.605      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1535 |                 1469 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  148 |                  416 |
+-----------------+----------------------+----------------------+


### Ada boosting

In [ ]:
np.random.seed(28)

ada_classifier = AdaBoostClassifier(random_state=28)

param_grid = {
    'n_estimators': [50, 80, 100, 120, 150],
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
        'algorithm': ['SAMME', 'SAMME.R']
}

grid_search = GridSearchCV(estimator=ada_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

grid_search.fit(X_downsampled, y_downsampled)

print("Best Hyperparameters:", grid_search.best_params_)

print("Best Score:", grid_search.best_score_)

Best Hyperparameters: {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 150}
Best Score: 0.6543524874585088


In [ ]:
ada_classifier = AdaBoostClassifier(n_estimators=150, learning_rate=1.0, algorithm='SAMME', random_state=28)

ada_classifier.fit(X_downsampled, y_downsampled)

y_pred_ada = ada_classifier.predict(X_test)

accuracy = round(metrics.accuracy_score(y_test, y_pred_ada) * 100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred_ada) * 100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred_ada, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred_ada)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  66.82 %
Misclassification rate of this model:  33.18 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.916     0.667     0.772      3004
           1      0.276     0.676     0.392       564

    accuracy                          0.668      3568
   macro avg      0.596     0.671     0.582      3568
weighted avg      0.815     0.668     0.712      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 2003 |                 1001 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  183 |                  381 |
+-----------------+----------------------+----------------------+


### Gradient Boosting

In [ ]:
np.random.seed(28)

gb_classifier = GradientBoostingClassifier(random_state=28)

param_grid = {
    'n_estimators': [50, 80, 100, 120, 150],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, None],
}

grid_search = GridSearchCV(estimator=gb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_downsampled, y_downsampled)

print("Best Hyperparameters:", grid_search.best_params_)

print("Best Score:", grid_search.best_score_)

Best Hyperparameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 150}
Best Score: 0.6476315538392781


In [ ]:
gb_classifier = GradientBoostingClassifier(n_estimators=150, learning_rate=0.05, max_depth = 3, random_state=28)
gb_classifier.fit(X_downsampled, y_downsampled)
y_pred = gb_classifier.predict(X_test)

accuracy = round(metrics.accuracy_score(y_test, y_pred)*100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred)*100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  62.22 %
Misclassification rate of this model:  37.78 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.913     0.610     0.731      3004
           1      0.249     0.690     0.366       564

    accuracy                          0.622      3568
   macro avg      0.581     0.650     0.548      3568
weighted avg      0.808     0.622     0.673      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1831 |                 1173 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  175 |                  389 |
+-----------------+----------------------+----------------------+


### Multilayer Perceptron

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_downsampled)
X_test_scaled = scaler.transform(X_test)


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam',alpha=0.0001, max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_downsampled)
y_pred = mlp.predict(X_test_scaled)

accuracy = round(metrics.accuracy_score(y_test, y_pred)*100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred)*100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred, digits=3))

conf_matrix = metrics.confusion_matrix(y_test, y_pred)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  58.77 %
Misclassification rate of this model:  41.23 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.877     0.594     0.708      3004
           1      0.205     0.557     0.299       564

    accuracy                          0.588      3568
   macro avg      0.541     0.575     0.504      3568
weighted avg      0.771     0.588     0.643      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1783 |                 1221 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  250 |                  314 |
+-----------------+----------------------+----------------------+


### k Nearest Neighbours

In [ ]:


np.random.seed(28)

# Initialize KNN classifier
knn_classifier = KNeighborsClassifier()

# Define parameter grid for KNN
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the model
grid_search.fit(X_downsampled, y_downsampled)


# Display best hyperparameters and best score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)


Best Hyperparameters: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}
Best Score: 0.5291263891050975


In [ ]:
# Use the best estimator from GridSearchCV to predict on test data
knn_classifier = KNeighborsClassifier(metric = 'manhattan', n_neighbors =  7, weights = 'uniform')
knn_classifier.fit(X_downsampled, y_downsampled)

y_pred_knn = knn_classifier.predict(X_test)

# Calculate accuracy and misclassification rate
accuracy = round(metrics.accuracy_score(y_test, y_pred_knn) * 100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred_knn) * 100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

# Print classification report
print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred_knn, digits=3))

# Print confusion matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred_knn)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  52.47 %
Misclassification rate of this model:  47.53 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.861     0.519     0.648      3004
           1      0.178     0.553     0.269       564

    accuracy                          0.525      3568
   macro avg      0.519     0.536     0.458      3568
weighted avg      0.753     0.525     0.588      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 1560 |                 1444 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  252 |                  312 |
+-----------------+----------------------+----------------------+


### Support Vector Machines

In [ ]:
np.random.seed(28)

g = [0.0001, 0.001, 0.01, 0.1]
hist = []
for val in g:
  clf = SVC(gamma=val)
  cross_val = cross_val_score(clf, X_downsampled, y_downsampled, cv=5)
  hist.append(np.mean(cross_val))
print(hist)
  #plt.plot([str(i) for i in g], hist)
  #plt.title('Cross Validations score for SVC')
  #plt.xlabel('gamma')
  #plt.ylabel('Accuracy')
  #plt.grid()
  #plt.show()

[0.49955207166853305, 0.4997762869151698, 0.49955207166853305, 0.49955207166853305]


In [ ]:
# Use the best estimator from GridSearchCV to predict on test data
svm_classifier = SVC(gamma=0.001)
svm_classifier.fit(X_downsampled, y_downsampled)

y_pred_svm = svm_classifier.predict(X_test)

# Calculate accuracy and misclassification rate
accuracy = round(metrics.accuracy_score(y_test, y_pred_svm) * 100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred_svm) * 100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

# Print classification report
print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred_svm, digits=3))

# Print confusion matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred_svm)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  84.19 %
Misclassification rate of this model:  15.81 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.842     1.000     0.914      3004
           1      0.000     0.000     0.000       564

    accuracy                          0.842      3568
   macro avg      0.421     0.500     0.457      3568
weighted avg      0.709     0.842     0.770      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 3004 |                    0 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  564 |                    0 |
+-----------------+----------------------+----------------------+


### Naive Bias

In [ ]:
np.random.seed(28)

# Initialize Naive Bayes classifier
nb_classifier = GaussianNB()

# Define parameter grid for Naive Bayes (even though GaussianNB has few hyperparameters)
param_grid = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5, n_jobs=-1)

# Fit the model
grid_search.fit(X_downsampled, y_downsampled)

# Display best hyperparameters and best score
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Hyperparameters: {'var_smoothing': 1e-09}
Best Score: 0.6124646603628622


In [ ]:
# Use the best estimator from GridSearchCV to predict on test data
nb_classifier = grid_search.best_estimator_
nb_classifier.fit(X_downsampled, y_downsampled)

y_pred_nb = nb_classifier.predict(X_test)

# Calculate accuracy and misclassification rate
accuracy = round(metrics.accuracy_score(y_test, y_pred_nb) * 100, 2)
print("Accuracy score of this model: ", accuracy, "%")

misclassification_rate = round(np.mean(y_test != y_pred_nb) * 100, 2)
print("Misclassification rate of this model: ", misclassification_rate, "%")

# Print classification report
print("\nReport card of this model: ")
print(metrics.classification_report(y_test, y_pred_nb, digits=3))

# Print confusion matrix
conf_matrix = metrics.confusion_matrix(y_test, y_pred_nb)
labels = ['Actual Negative', 'Actual Positive']
columns = ['Predicted Negative', 'Predicted Positive']
confusion_table = tabulate(conf_matrix, headers=columns, showindex=labels, tablefmt='grid')
print("Confusion Matrix:")
print(confusion_table)

Accuracy score of this model:  66.62 %
Misclassification rate of this model:  33.38 %

Report card of this model: 
              precision    recall  f1-score   support

           0      0.898     0.681     0.774      3004
           1      0.257     0.589     0.358       564

    accuracy                          0.666      3568
   macro avg      0.578     0.635     0.566      3568
weighted avg      0.797     0.666     0.709      3568

Confusion Matrix:
+-----------------+----------------------+----------------------+
|                 |   Predicted Negative |   Predicted Positive |
+=================+======================+======================+
| Actual Negative |                 2045 |                  959 |
+-----------------+----------------------+----------------------+
| Actual Positive |                  232 |                  332 |
+-----------------+----------------------+----------------------+
